<a href="https://colab.research.google.com/github/THEONE2456/Repositorio_de_la_clase_de_vis-n_Artificial./blob/main/Copia_de_Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Subir kaggle.json
from google.colab import files
files.upload()  # Selecciona tu kaggle.json

!mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d itsahmad/indoor-scenes-cvpr-2019
!unzip -q indoor-scenes-cvpr-2019.zip -d dataset

DATASET_PATH = "dataset/indoorCVPR_09/Images"
!ls "$DATASET_PATH" | head -n 10  # ver algunas clases

mv: cannot stat 'kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open indoor-scenes-cvpr-2019.zip, indoor-scenes-cvpr-2019.zip.zip or indoor-scenes-cvpr-2019.zip.ZIP.
ls: cannot access 'dataset/indoorCVPR_09/Images': No such file or directory


Generadores y partición (80/20 sin mover físicamente)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 224
BATCH = 32

gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_ds = gen.flow_from_directory(DATASET_PATH, target_size=(IMG_SIZE, IMG_SIZE),
                                   batch_size=BATCH, subset='training')
val_ds   = gen.flow_from_directory(DATASET_PATH, target_size=(IMG_SIZE, IMG_SIZE),
                                   batch_size=BATCH, subset='validation')
num_classes = train_ds.num_classes
class_names = list(train_ds.class_indices.keys())

Found 12522 images belonging to 67 classes.
Found 3098 images belonging to 67 classes.


Modelo (MobileNetV2

In [ ]:
base = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE,IMG_SIZE,3),
                                         include_top=False, weights='imagenet')
base.trainable = False

from tensorflow.keras import layers, models
model = models.Sequential([
  base,
  layers.GlobalAveragePooling2D(),
  layers.Dropout(0.2),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

# Guardado (formato nuevo recomendado)
model.save("indoor_scenes.keras")

Epoch 1/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 85s 196ms/step - accuracy: 0.3356 - loss: 2.7463 - val_accuracy: 0.6159 - val_loss: 1.3404
Epoch 2/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 66s 170ms/step - accuracy: 0.6577 - loss: 1.1631 - val_accuracy: 0.6611 - val_loss: 1.1683
Epoch 3/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 87s 183ms/step - accuracy: 0.7273 - loss: 0.9063 - val_accuracy: 0.6578 - val_loss: 1.1608
Epoch 4/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 71s 181ms/step - accuracy: 0.7689 - loss: 0.7626 - val_accuracy: 0.6704 - val_loss: 1.1517
Epoch 5/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 77s 167ms/step - accuracy: 0.7885 - loss: 0.6718 - val_accuracy: 0.6624 - val_loss: 1.1778
Epoch 6/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 67s 170ms/step - accuracy: 0.8154 - loss: 0.6031 - val_accuracy: 0.6679 - val_loss: 1.1614
Epoch 7/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 66s 167ms/step - accuracy: 0.8306 - loss: 0.5292 - val_accuracy: 0.6775 - val_loss: 1.1963
Epoch 8/10
392/392 ━━━━━━━━━━━━━━━━━━━━ 66s 170ms/step - accuracy: 0.8546 - loss: 0

Conversión a TensorFlow Lite + etiquetas

In [ ]:
# TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("indoor_scenes.tflite", "wb").write(tflite_model)

# Etiquetas (una por línea para Android)
with open("labels.txt", "w") as f:
    for name in class_names:
        f.write(name + "\n")

# Descarga a tu PC
from google.colab import files
files.download("indoor_scenes.tflite")
files.download("labels.txt")

Saved artifact at '/tmp/tmps3l9swrg'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_314')
Output Type:
  TensorSpec(shape=(None, 67), dtype=tf.float32, name=None)
Captures:
  137697534393616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137697534393424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137697534390928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137697534391696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137697534391504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137697534391120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137697534392656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137697534381520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137697534380368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137697534391312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137697534

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Comprobación rápida con una imagen

In [ ]:
# Subir una imagen y predecir
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

uploaded = files.upload()
img_path = list(uploaded.keys())[0]
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
arr = np.expand_dims(image.img_to_array(img)/255.0, axis=0)

pred = model.predict(arr)
i = int(np.argmax(pred))
conf = float(np.max(pred))*100
print(f"Predicción: {class_names[i]} ({conf:.2f}%)")

Saving descarga.jpg to descarga.jpg


NameError: name 'IMG_SIZE' is not defined

crear y guardar el archivo

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define IMG_SIZE and num_classes (assuming they are still needed for model definition)
# You might need to run the previous cells to get the correct values if the runtime was reset
IMG_SIZE = 224 # Example value, replace with actual value if needed
num_classes = 67 # Example value, replace with actual value if needed

# Define the model architecture (copying from cell U5UNtTodGcby)
base = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE,IMG_SIZE,3),
                                         include_top=False, weights='imagenet')
base.trainable = False

model = models.Sequential([
  base,
  layers.GlobalAveragePooling2D(),
  layers.Dropout(0.2),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])


# 📌 Convertir el modelo entrenado a TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 📌 Guardar el modelo convertido
with open("modelo_indoor.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Modelo convertido y guardado como modelo_indoor.tflite")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Saved artifact at '/tmp/tmpyx_8_a4q'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 67), dtype=tf.float32, name=None)
Captures:
  139348280414608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139348280417872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139348280420560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139348280420176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139348280419024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139348280420752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139348280419216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139348280421328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139348280420944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139348280418832: TensorSpec(sh

Exportar el modelo desde Colab

In [ ]:
from google.colab import files
files.download("modelo_indoor.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>